# Sandbox

In [147]:
import requests
import pandas as pd
from datetime import datetime

In [148]:
import math

In [149]:
from google.transit import gtfs_realtime_pb2

In [169]:
via_trips = pd.read_csv(os.path.join('gtfs', 'viarail', 'trips.txt'))

In [170]:
amtrak_trips = pd.read_csv(os.path.join('gtfs', 'amtrak', 'trips.txt'))

In [172]:
via_trips

,route_id,service_id,trip_id,shape_id,trip_short_name,trip_headsign,direction_id
0,119-341,498,498,498,85,London,0
1,119-341,499,499,499,87,Sarnia,0
2,119-93,494,494,494,80,Toronto,1
3,119-618,495,495,495,72,Toronto,1
4,119-618,496,496,496,76,Toronto,1
...,...,...,...,...,...,...,...
91,119-58,474,474,474,650,Kingston,0
92,226-119,475,475,475,64,Montréal,1
93,617-119,478,478,478,44,Ottawa,1
94,617-119,479,479,479,46,Ottawa,1


In [75]:
condition = trips['trip_id'] == 517

In [76]:
trips[condition]

,route_id,service_id,trip_id,shape_id,trip_short_name,trip_headsign,direction_id
44,119-618,517,517,517,79,Windsor,0


In [25]:
via_feed_url = 'https://asm-backend.transitdocs.com/gtfs/via'
amtrak_feed_url = 'https://asm-backend.transitdocs.com/gtfs/amtrak'

response = requests.get(via_feed_url)
if response.status_code == 200:
    via_feed = gtfs_realtime_pb2.FeedMessage()
    via_feed.ParseFromString(response.content)

    # Process the GTFS Realtime data
    # You can access various entities in the feed, such as feed.entity, feed.header, etc.
else:
    print('Error retrieving GTFS Realtime data:', response.status_code)
    
response = requests.get(amtrak_feed_url)
if response.status_code == 200:
    amtrak_feed = gtfs_realtime_pb2.FeedMessage()
    amtrak_feed.ParseFromString(response.content)

    # Process the GTFS Realtime data
    # You can access various entities in the feed, such as feed.entity, feed.header, etc.
else:
    print('Error retrieving GTFS Realtime data:', response.status_code)

In [153]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two GPS coordinates using the Haversine formula.
    Args:
        lat1 (float): Latitude of the first coordinate.
        lon1 (float): Longitude of the first coordinate.
        lat2 (float): Latitude of the second coordinate.
        lon2 (float): Longitude of the second coordinate.
    Returns:
        float: Distance between the two coordinates in meters.
    """
    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Radius of the Earth in meters
    earth_radius = 6371000

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c

    return distance

In [213]:
via_feed_url = 'https://asm-backend.transitdocs.com/gtfs/via'
response = requests.get(via_feed_url)
if response.status_code == 200:
    via_feed = gtfs_realtime_pb2.FeedMessage()
    via_feed.ParseFromString(response.content)

In [163]:
amtrak_feed_url = 'https://asm-backend.transitdocs.com/gtfs/amtrak'
response = requests.get(amtrak_feed_url)
if response.status_code == 200:
    amtrak_feed = gtfs_realtime_pb2.FeedMessage()
    amtrak_feed.ParseFromString(response.content)

In [268]:
exo_trips_url = 'https://opendata.exo.quebec/ServiceGTFSR/TripUpdate.pb?token=9VZEYEXS3C&agency=TRAINS'
exo_vehicle_url = 'https://opendata.exo.quebec/ServiceGTFSR/VehiclePosition.pb?token=9VZEYEXS3C&agency=TRAINS'

response = requests.get(exo_trips_url)
if response.status_code == 200:
    exo_trips = gtfs_realtime_pb2.FeedMessage()
    exo_trips.ParseFromString(response.content)
else:
    print(response.status_code)
    
response = requests.get(exo_vehicle_url)
if response.status_code == 200:
    exo_vehicles = gtfs_realtime_pb2.FeedMessage()
    exo_vehicles.ParseFromString(response.content)
else:
    print(response.status_code)

In [287]:
exo_trips = pd.read_csv(os.path.join('gtfs', 'exo', 'trips.txt'))
exo_routes = pd.read_csv(os.path.join('gtfs', 'exo', 'routes.txt'))

In [290]:
exo_trips.loc[exo_trips['trip_id'] == '4084038-TRAIN-H23-Blocks-Semaine-09']

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,trip_short_name,wheelchair_accessible,bikes_allowed
169,5,TRAIN-H23-Blocks-Semaine-09,4084038-TRAIN-H23-Blocks-Semaine-09,Lucien-L'Allier,0,50077,84,0,1


In [300]:
for exo_train in exo_vehicles.entity:
    #print(exo_train)
    
    if exo_train.HasField('vehicle'):
        vehicle = exo_train.vehicle
        print('Route: ' + vehicle.trip.route_id)
        print('Vehicle ID: ' + vehicle.vehicle.id)
        print('Trip ID: ' + vehicle.trip.trip_id)
        
        # resolve train no. from trips table
        try:
            condition = (exo_trips['trip_id'] == vehicle.trip.trip_id)
            print('Train No: ' + str(exo_trips.loc[condition, 'trip_short_name'].values[0]))
            print('Destination: ' + exo_trips.loc[condition, 'trip_headsign'].values[0])
        except:
            pass
        
        # resolve route name from routes table
        try:
            # because trip_id is numeric, pandas stores it as integer
            condition = (exo_routes['route_id'] == int(vehicle.trip.route_id))
            print('Route Name: ' + exo_routes.loc[condition, 'route_long_name'].values[0])
        except:
            pass
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        print('Distance from Gracey Side Rd: ' + str(round(calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)/10)/100) + ' km')
        print()
        

Route: 1
Vehicle ID: 3006
Trip ID: 4083546-TRAIN-H23-Blocks-Semaine-09
Train No: 13
Destination: Vaudreuil
Route Name: Vaudreuil–Hudson
Longitude: 45.401641845703125
Latitude: -73.958740234375
Distance from Gracey Side Rd: 766.27 km

Route: 5
Vehicle ID: 3011
Trip ID: 4084038-TRAIN-H23-Blocks-Semaine-09
Train No: 84
Destination: Lucien-L'Allier
Route Name: Candiac
Longitude: 45.42442321777344
Latitude: -73.6572494506836
Distance from Gracey Side Rd: 788.88 km



In [269]:
exo_train_list = []
for exo_trip in exo_trips.entity:
    #print(exo_trip)
    
    if exo_trip.HasField('trip_update'):
        trip_id = exo_trip.trip_update.trip.trip_id
        train_id = exo_trip.id
        exo_train_list.append({'train_id': train_id, 'trip_id': trip_id})
        
        print('Train ' + train_id + ' has trip id ' + trip_id)

print(exo_train_list)
print()

for exo_train in exo_vehicles.entity:
    #print(exo_train)
    
    if exo_train.HasField('vehicle'):
        vehicle = exo_train.vehicle
        print('Route: ' + vehicle.trip.route_id)
        print('Vehicle ID: ' + vehicle.vehicle.id)
        print('Trip ID: ' + vehicle.trip.trip_id)
        for exo_train in exo_train_list:
            if exo_train['trip_id'] == vehicle.trip.trip_id:
                print('Train No: ' + exo_train['train_id'])
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        print('Distance from Gracey Side Rd: ' + str(round(calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)/10)/100) + ' km')
        print()
        

Train 84 has trip id 4084038-TRAIN-H23-Blocks-Semaine-09
Train 184 has trip id 4084012-TRAIN-H23-Blocks-Semaine-09
Train 13 has trip id 4083546-TRAIN-H23-Blocks-Semaine-09
Train 22 has trip id 4083565-TRAIN-H23-Blocks-Semaine-09
[{'train_id': '84', 'trip_id': '4084038-TRAIN-H23-Blocks-Semaine-09'}, {'train_id': '184', 'trip_id': '4084012-TRAIN-H23-Blocks-Semaine-09'}, {'train_id': '13', 'trip_id': '4083546-TRAIN-H23-Blocks-Semaine-09'}, {'train_id': '22', 'trip_id': '4083565-TRAIN-H23-Blocks-Semaine-09'}]

Route: 1
Vehicle ID: 3006
Trip ID: 4083546-TRAIN-H23-Blocks-Semaine-09
Train No: 13
Longitude: 45.401641845703125
Latitude: -73.958740234375
Distance from Gracey Side Rd: 766.27 km

Route: 5
Vehicle ID: 3011
Trip ID: 4084038-TRAIN-H23-Blocks-Semaine-09
Train No: 84
Longitude: 45.42442321777344
Latitude: -73.6572494506836
Distance from Gracey Side Rd: 788.88 km



In [208]:
datetime.now()

datetime.datetime(2023, 5, 30, 6, 19, 57, 672611)

In [249]:
train_list = []
for via_train in via_feed.entity:
    #print(via_train)
    
    if via_train.HasField('vehicle'):
        vehicle = via_train.vehicle
        print(vehicle)
        
        print('Trip Start Date: ' + vehicle.trip.start_date)
        timedelta = datetime.now() - datetime.fromtimestamp(vehicle.timestamp)
        print('Updated: ' + datetime.fromtimestamp(vehicle.timestamp).strftime('%Y-%m-%d %H:%M:%S') + ', ' + str(timedelta) + ' ago.')
        
        
        try:
            # because trip_id is numeric, pandas stores it as integer
            condition = (via_trips['trip_id'] == int(vehicle.trip.trip_id)) & (via_trips['route_id'] == vehicle.trip.route_id)
            print('Train ID: ' + via_trips.loc[condition, 'trip_short_name'].values[0])
            print('Destination: ' + via_trips.loc[condition, 'trip_headsign'].values[0])
        except:
            pass
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        dst = calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)
        print('Distance from Gracey Side Rd: ' + str(round(dst/10)/100) + ' km')
        
        # https://asm.transitdocs.com/train/2023/5/29/V/63
        print(f'https://asm.transitdocs.com/train/{vehicle.trip.start_date[0:4]}/{vehicle.trip.start_date[4:6]}/{vehicle.trip.start_date[6:8]}/V/{vehicle.vehicle.id[15:]}')
        
        print()

trip {
  trip_id: "112"
  route_id: "8-119"
  direction_id: 0
  start_date: "20230526"
  schedule_relationship: SCHEDULED
}
vehicle {
  id: "2023-05-26_VIA_2"
}
position {
  latitude: 46.2906
  longitude: -80.844
  speed: 16.1113224
}
current_stop_sequence: 65
stop_id: "69"
current_status: IN_TRANSIT_TO
timestamp: 1685442099

Trip Start Date: 20230526
Updated: 2023-05-30 06:21:39, 3:59:41.942175 ago.
Train ID: 2
Destination: Toronto
Longitude: 46.29059982299805
Latitude: -80.84400177001953
Distance from Gracey Side Rd: 462.36 km
https://asm.transitdocs.com/train/2023/05/26/V/2

trip {
  trip_id: "483"
  route_id: "226-119"
  direction_id: 0
  start_date: "20230530"
  schedule_relationship: SCHEDULED
}
vehicle {
  id: "2023-05-30_VIA_51"
}
position {
  latitude: 45.4997
  longitude: -73.5638
  speed: 0
}
current_stop_sequence: 1
stop_id: "226"
current_status: IN_TRANSIT_TO
timestamp: 1685442150

Trip Start Date: 20230530
Updated: 2023-05-30 06:22:30, 3:58:50.944168 ago.
Train ID: 51
Des

In [225]:
'; '.join(train_list)

'Train No: VIA 2; Train No: VIA 70'

In [193]:
for amtk_train in amtrak_feed.entity:
    #print(entity)
    
    if amtk_train.HasField('vehicle'):
        vehicle = amtk_train.vehicle
        #print(vehicle)
        
        print('Trip Start Date: ' + vehicle.trip.start_date)
        print('Updated: ' + datetime.fromtimestamp(vehicle.timestamp).strftime('%Y-%m-%d %H:%M:%S'))
        
        #condition = (amtrak_trips['trip_id'] == vehicle.trip.trip_id) & (amtrak_trips['route_id'] == vehicle.trip.route_id)
        #print('Train ID: ' + amtrak_trips.loc[condition, 'trip_short_name'].values[0])
        #print('Destination: ' + amtrak_trips.loc[condition, 'trip_headsign'].values[0])
        
        print('Longitude: ' + str(vehicle.position.latitude))
        print('Latitude: ' + str(vehicle.position.longitude))
        print('Distance from Gracey Side Rd: ' + str(round(calculate_distance(42.305618, -82.504051, vehicle.position.latitude, vehicle.position.longitude)/1000)) + ' km')
        print()

Trip Start Date: 20230529
Updated: 2023-05-29 23:57:08
Longitude: 43.19765853881836
Latitude: -70.87969970703125
Distance from Gracey Side Rd: 954 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:57:06
Longitude: 37.22612762451172
Latitude: -120.2425537109375
Distance from Gracey Side Rd: 3247 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:34
Longitude: 37.293643951416016
Latitude: -120.32691192626953
Distance from Gracey Side Rd: 3251 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:07
Longitude: 37.93414306640625
Latitude: -121.3546142578125
Distance from Gracey Side Rd: 3310 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:02
Longitude: 41.34831619262695
Latitude: -71.87882232666016
Distance from Gracey Side Rd: 886 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:56:57
Longitude: 40.326637268066406
Latitude: -74.61068725585938
Distance from Gracey Side Rd: 695 km

Trip Start Date: 20230529
Updated: 2023-05-29 23:43:00
Longitude: 40.752464294433594
Lat

In [301]:
exo_file = 'exo_credentials.txt'

In [303]:
with open(exo_file, 'r') as f:
    line = f.read()

In [307]:
str(line.strip()).strip()

'9VZEYEXS3C'